In [6]:
import requests
import pyarabic.araby as araby # for processing arabic 
import re

In [7]:
#udpipe stuff : i love reproducable code  !  
def udpipe(string) :
    # Prepping String 
    words  = re.findall(u'[\u0600-\u06FF]+', string) #getting arabic characters 
    data = ' '.join(words)
    
    pipe_base_url = 'http://lindat.mff.cuni.cz/services/udpipe/api/process?tokenizer&tagger' 
    attributes = {} 
    attributes['model'] = 'arabic-ud-2.0-170801'
    attributes['data'] = data

    data = requests.get(pipe_base_url , attributes)
    result = data.json()['result'].split('\n')
    udpipe_results = [re.findall(u'[\u0600-\u06FF]+', i) for i in result] # cleaning 
    udpipe_results = [i for i in udpipe_results if i !=[]]
    
    return udpipe_results

def udpipe_reconstruct(original_text) : 
    udpipe_results = udpipe(original_text)
    for i in range(1, len(udpipe_results)):
        if len(udpipe_results[i]) == 1 : # composite verbs get the next two words
            original_text =original_text.replace(udpipe_results[i][0] , '{} {}'.format(udpipe_results[i+1][1],udpipe_results[i+2][1]))
        else :
            original_text =original_text.replace(udpipe_results[i][0] , udpipe_results[i][1])
    original_text = araby.strip_tashkeel(original_text)
    return original_text


def master_reconstruct_input(text_input , input_type): 
    # type 0: none-rule just do just do udpipe_reconstruction
    # type 1 : fix the rule + do udpipe_reconstruction - no need for this here 
    if input_type == 0:
        return udpipe_reconstruct(text_input)

In [8]:
with open('agents.all.ar.txt', 'r') as agent_file : 
    agents = agent_file.read()

agents_file = agents.split('\n')
agents = [i for i in agents_file if i != '' and i[0] !='#']

In [2]:
agents[:20]

['ARMED_SEPARATIST [~REB]انفصالي مسلحي',
 'ARMED_SEPARATIST_MOVEMENT [~REB]حركة انفصالية مسلحة',
 'ARMY_FOR_THE_LIBERATION_  [~REB]جيش للتحرير',
 'ETHNIC_SEPARATISTS [~REB] # ?انفصاليين عرقيين',
 'FRONT_FOR_THE_LIBERATION_  [~REB]جبهة التحرير',
 'LIBERATION_ARMY_  [~REB]جيش التحرير',
 'LIBERATION_FORCES_  [~REB]قوى التحرير',
 'LIBERATION_FRONT_  [~REB]جبهة التحرير',
 'LIBERATION_MOVEMENT  [~OPP]حركة تحرير',
 'POPULAR_MOVEMENT_FOR_THE_LIBERATION_  [~REB]حركة جماهيرية من أجل التحرير',
 'PRO_INDEPENDENCE_ACTIVIST [~OPP] # ? REB SEEMED WRONG ناشط مؤيد للاستقلال',
 'REBEL_SEPARATIST [~REB] انفصاليين متمردين',
 'SEPARATIST [~REB]انفصالي',
 'SEPARATIST_GROUP [~REB]مجموعة انفصالية',
 'SEPARATIST_GUERRILLA [~REB]حرب عصابات انفصالية',
 'SEPARATIST_LEADER [~REB]قائد انفصالي',
 'SEPARATIST_MILITANT [~REB]مقاتل انفصالي',
 'SEPARATIST_MOVEMENT [~REB]حركة انفصالية',
 'SEPARATIST_REBEL [~REB]متمردين انفصاليين',
 'INTERNATIONAL_TERRORIST [~IMG]إرهابي دولي']

In [9]:
arabic_only_agents = [x.split('[') for x in agents]
arabic_only_agents=  [i[1] if len(i) > 1 else i[0] for i in arabic_only_agents ]
arabic_only_agents = ['[' + i for i in arabic_only_agents]
# arabic_only_agents

In [10]:
#udpipe stuff tho
arabic_only_agents_udpipe = [] 
for i in arabic_only_agents : 
    arabic_only_agents_udpipe.append(master_reconstruct_input(i,0))

In [11]:
reconstructed_agents = []
for i in arabic_only_agents_udpipe:
    agent = '_'.join(re.findall(r'[\u0600-\u06FF]+', i))
    code =i.split(']')[0] + ']'
    result = '{} {}'.format(agent,code)
    reconstructed_agents.append(result)
#     print(result)

In [12]:
reconstructed_agents
with open('agents.all.ar_v3.txt' ,'w') as output_file: 
    output_file.write('\n'.join(reconstructed_agents))

## Synsets 

In [17]:
lines = []
with open('synsets.ar.txt' , 'r') as finput : 
    for i in finput  :
        lines.append(i)
lines[:10]

['&CURRENCY\n',
 '\n',
 '+درهم_مغربي_\n',
 '+فرنك_كونغولي_\n',
 '+ناكفا_\n',
 '+فرنك_سويسري_\n',
 '+نيرة_نيجيرية_\n',
 '+تالا_\n',
 '+روبية_نيبالية_\n',
 '+يورو_\n']

In [18]:
lines2 = [] 
for i in lines : 
    if i[0] == '&':
        lines2.append(i)
    else :
        lines2.append(master_reconstruct_input(i,0))

lines2[:10]

['&CURRENCY\n',
 '\n',
 '+در هو_مغربي_\n',
 '+فرنك_كونغولي_\n',
 '+ناكفا_\n',
 '+فرنك_سويسري_\n',
 '+نيرة_نيجيرية_\n',
 '+تالا_\n',
 '+روبية_نيبالية_\n',
 '+يورو_\n']

In [19]:
with open('synsets.ar.v2.txt', 'w') as output :
    for i in lines2: 
        output.write(i)
